In [0]:
# MAGIC %load_ext autoreload
# MAGIC %autoreload 2

In [0]:
import sys 
import os 
import pyspark.sql.functions as f 
sys.path.append(os.path.abspath('../..'))

In [0]:
from data_loaders.data_load_autoloader import load_raw_data_with_schema_evolution

In [0]:
CATALOG = "pei"
STORAGE_BASE = "abfss://retail@peiretailstorageaccount.dfs.core.windows.net/landing"

In [0]:
product_load_options = {
    "badRecordsPath": f"{STORAGE_BASE}/products",
    "cloudFiles.schemaLocation": f"/Volumes/{CATALOG}/default/checkpoint_volume/raw_products_schema_checkpoint"
}

In [0]:
try:
  products_df = load_raw_data_with_schema_evolution(spark_session=spark, 
                                          source_path=f"{STORAGE_BASE}/products/*.csv",
                                          file_format="csv",
                                          additional_options=product_load_options)


  ( products_df
    .withColumn("file_path", f.col("_metadata.file_path")) 
    .withColumn("ingestion_timestamp", f.current_timestamp()) 
    .writeStream
    .option("checkpointLocation", f'/Volumes/{CATALOG}/default/checkpoint_volume/raw_products_file_checkpoint') 
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable(f"{CATALOG}.bronze.raw_products")
  )  
except Exception as e: 
  print(f"FAILED: Products Ingestion. Error: {str(e)}")